In [6]:
import chess
import chess.pgn
import chess.engine
import torch    
import torch.nn as nn
import torch.optim as optim
import time
import chess.svg
from IPython.display import SVG

# Load the PGN dataset
pgn_file = "lichess_db_standard_rated_2016-07.pgn"
pgn = open(pgn_file, "r")
game = chess.pgn.read_game(pgn)

# Initialize the chess engine (Stockfish)
engine = chess.engine.SimpleEngine.popen_uci("stockfish-windows-x86-64-avx2")

chess.Board()

    # Get the best move from the chess engine
result = engine.play(board, chess.engine.Limit(time=2.0))
best_move = result.move



# Iterate through the moves in the game
board = game.board()
for move in game.mainline_moves():
 
    b = chess.Board()

svg = chess.svg.board(b, size=250)
display(SVG(data=svg))

    # Get the best move from the chess engine
result = engine.play(board, chess.engine.Limit(time=2.0))
best_move = result.move

    # Make the best move on the board
board.push(best_move)



# Close the chess engine
engine.quit()


NameError: name 'board' is not defined

In [ ]:
board = chess.Board()


In [ ]:
board.turn

In [ ]:
board.turn == chess.WHITE

In [ ]:
board

In [ ]:
print(str(board))

In [ ]:
chess.C1

In [ ]:
board.piece_at(chess.C1)

In [ ]:
str(board.piece_at(chess.C1))

In [ ]:
list(board.legal_moves)

In [ ]:
move = list(board.legal_moves)[0]

In [ ]:
move

In [ ]:
move.uci()

In [ ]:
board.san(move)

In [ ]:
import random

In [ ]:
def random_player(board):
    move = random.choice(list(board.legal_moves))
    return move.uci()

In [ ]:
random_player(board)


In [ ]:
for i in range(10):
    print(random_player(board))

In [ ]:
import time
from IPython.display import display, HTML, clear_output

In [ ]:
def who(player):
    return "White" if player == chess.WHITE else "Black"

In [ ]:
def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

In [ ]:
def play_game(player1, player2, visual="svg", pause=0.1):
    """
    playerN1, player2: functions that takes board, return uci move
    visual: "simple" | "svg" | None
    """
    use_svg = (visual == "svg")
    board = chess.Board()
    try:
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                uci = player1(board)
            else:
                uci = player2(board)
            name = who(board.turn)
            board.push_uci(uci)
            board_stop = display_board(board, use_svg)
            html = "<b>Move %s %s, Play '%s':</b><br/>%s" % (
                       len(board.move_stack), name, uci, board_stop)
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
    except KeyboardInterrupt:
        msg = "Game interrupted!"
        return (None, msg, board)
    result = None
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        result = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: 5-fold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    if visual is not None:
        print(msg)
    return (result, msg, board)

In [ ]:
play_game(random_player, random_player)

In [ ]:
def human_player(board):
    display(board)
    uci = get_move("%s's move [q to quit]> " % who(board.turn))
    legal_uci_moves = [move.uci() for move in board.legal_moves]
    while uci not in legal_uci_moves:
        print("Legal moves: " + (",".join(sorted(legal_uci_moves))))
        uci = get_move("%s's move[q to quit]> " % who(board.turn))
    return uci


In [ ]:
def get_move(prompt):
    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()
    try:
        chess.Move.from_uci(uci)
    except:
        uci = None
    return uci

In [ ]:
counts = {None: 0, True: 0, False: 0}
for i in range(10):
    result, msg, board = play_game(random_player, random_player, visual=None)
    counts[result] += 1
    print(counts)
counts

In [ ]:
def player1(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = staticAnalysis(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()


In [ ]:
board = chess.Board()

board.pieces(chess.ROOK, True)

In [ ]:
list(board.pieces(chess.ROOK, True))


In [ ]:
len(board.pieces(chess.ROOK, True))

In [ ]:
def staticAnalysis(board, move, my_color):
    score = 0
    ## Check some things about this move:
    # score += 10 if board.is_capture(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    return score

In [ ]:
play_game(player1, random_player)

In [ ]:
def staticAnalysis(board, move, my_color):
    score = random.random()
    ## Check some things about this move:
    # score += 10 if board.is_capture(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    return score

In [ ]:
play_game(player1, random_player)